In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [2]:
#Import CSV from WeatherPy to create a DataFrame
df = pd.read_csv (r'../WeatherPy/output_data/cities.csv')
df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,busselton,-33.65,115.33,8.89,90,4,4.83,AU,1596481791
1,1,ibitinga,-21.76,-48.83,27.00,36,0,2.10,BR,1596482077
2,2,shingu,33.73,135.98,22.50,84,35,2.42,JP,1596482077
3,3,vanavara,60.34,102.28,14.04,65,77,2.17,RU,1596482078
4,4,punta arenas,-53.15,-70.92,8.00,68,20,7.20,CL,1596482078
...,...,...,...,...,...,...,...,...,...,...
557,557,kavaratti,10.57,72.64,28.05,78,100,10.62,IN,1596482159
558,558,kropotkin,45.44,40.58,19.55,50,0,2.72,RU,1596482159
559,559,te anau,-45.42,167.72,5.72,87,100,1.28,NZ,1596482159
560,560,fukue,32.69,128.84,26.99,83,98,5.29,JP,1596482159


# Humidity Heatmap

In [3]:
# Store latitude and longitude
cities = df[["Lat", "Lng"]]

# Store cities
humidity = df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(cities, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
#Create new DataFrame fitting weather criteria
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

ideal_df = df.loc[(df["Max Temp"] > 21) & (df["Max Temp"] < 26) & (df["Cloudiness"] == 0) & (df["Wind Speed"] < 10), :]
ideal_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,arraial do cabo,-22.97,-42.03,25.00,60,0,4.60,BR,1596482061
12,12,lagoa,39.05,-27.98,23.71,84,0,6.81,PT,1596482074
27,27,rikitea,-23.12,-134.97,23.06,82,0,7.76,PF,1596481789
28,28,lompoc,34.64,-120.46,23.00,56,0,6.70,US,1596482080
107,107,goderich,43.75,-81.72,22.78,69,0,1.34,CA,1596482090
113,113,cefalu,38.04,14.02,25.56,89,0,1.79,IT,1596482091
135,135,caravelas,-17.71,-39.25,23.08,62,0,4.92,BR,1596482094
137,137,minas,-34.38,-55.24,22.78,73,0,2.10,UY,1596482094
142,142,namatanai,-3.67,152.43,25.84,80,0,2.43,PG,1596482095
143,143,kabalo,-6.05,26.92,25.07,29,0,2.05,CD,1596482095
